In [2]:
# test_single_sample.ipynb

import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from torchvision import transforms as T
from model import DenoiseModel,get_noise_schedule   # Import your model definition
# from diffusion_utils import get_noise_schedule  # Your noise schedule function

# --- Setup ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DenoiseModel().to(device)
model.load_state_dict(torch.load("denoise_model.pth", map_location=device))
model.eval()

# --- Diffusion Settings ---
T_steps = 1000
betas, alphas, alpha_bars = get_noise_schedule(T=T_steps)

# --- Start from random noise ---
batch_size = 1
x_t = torch.randn((batch_size, 3, 64, 64)).to(device)

# --- Reverse Process ---
with torch.no_grad():
    for t in reversed(range(1, T_steps)):
        t_batch = torch.full((batch_size,), t, device=device, dtype=torch.long)

        predicted_noise = model(x_t, t_batch)

        alpha = alphas[t]
        alpha_bar = alpha_bars[t]
        beta = betas[t]

        mean = (1 / alpha.sqrt()) * (x_t - ((1 - alpha) / (1 - alpha_bar).sqrt()) * predicted_noise)

        if t > 1:
            noise = torch.randn_like(x_t)
            x_t = mean + beta.sqrt() * noise
        else:
            x_t = mean

# --- Convert to image ---
img = x_t[0].detach().cpu()
img = (img + 1) / 2  # if normalized to [-1, 1]
img = torch.clamp(img, 0.0, 1.0)
img = T.ToPILImage()(img)

# --- Display ---
plt.imshow(img)
plt.axis("off")
plt.title("Generated Sample Image")
plt.show()



RuntimeError: CUDA error: unspecified launch failure
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
